In [1]:
import cadquery as cq

In [2]:
def create_half_torus(thickness: float, radius: float):
    return (
        cq.Workplane("XZ", origin=(radius, radius, 0))
        .circle(thickness / 2)
        .revolve(180, (-radius, 0, 0), (-radius, -1, 0))
        .rotateAboutCenter((0, 1, 0), 90)
        .translate((0, -radius, 0))
    )

create_half_torus(1, 3)

In [3]:
def create_headband_arc(
    radius: float,
    thickness: float,
    extension: float,
    height: float,
):
    inner_radius = radius - thickness
    wp = (
        cq.Workplane("XY")
        .moveTo(radius, 0)
        .threePointArc((0, radius), (-radius, 0))
        .vLineTo(-extension)
        .hLineTo(-inner_radius)
        .vLineTo(0)
        .threePointArc((0, inner_radius), (inner_radius, 0))
        .vLineTo(-extension)
        .hLineTo(radius)
        .vLineTo(0)
        .close()
    )
    return wp.extrude(height)


band = create_headband_arc(
    radius=8.5,
    thickness=0.6,
    extension=3,
    height=3.5,
)
loop = create_half_torus(0.3, 1)
for face in band.faces("<Y").vals():
    face_loop = loop.translate(face.Center())
    band = band.union(face_loop)
band

Overwriting auto display for cadquery Workplane and Shape


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [4]:
def create_screw_mount(
    hole_diameter: float,
    hole_padding: float,
    depth: float,
):
    inner_radius = hole_diameter / 2
    outer_radius = inner_radius + hole_padding
    outer_cylinder = cq.Workplane("XZ").circle(outer_radius).extrude(depth)
    inner_cylinder = cq.Workplane("XZ").circle(inner_radius).extrude(depth)
    return outer_cylinder.cut(inner_cylinder)


create_screw_mount(1, 0.5, 1)

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [5]:
def create_strap_guide(height: float, width: float, depth: float, strap_clearance: float):
    padding = depth - strap_clearance
    assert padding >= 0.1
    outer_rect = (
        cq.Workplane("YZ")
        .rect(strap_clearance + padding * 2, height + padding * 2, centered=(True, True))
        .extrude(width)
    )
    inner_rect = (
        cq.Workplane("YZ")
        .rect(strap_clearance + padding * 2, height, centered=(True, True))
        .extrude(width)
        .translate((0, padding, 0))
    )
    y_offset = outer_rect.faces(">Y").val().Center().y
    return outer_rect.cut(inner_rect).translate((0, -y_offset, 0)) 


create_strap_guide(2, 1, 0.4, 0.2)

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [6]:
from typing import Tuple


def create_arc(
    width: float,
    height: float,
    head_radius: float,
    head_radius_penetration: float,
):
    arc_box = cq.Workplane("XZ").box(
        width, height, head_radius_penetration, centered=(True, True, False)
    )
    arc_cyclinder = (
        cq.Workplane("XY", origin=(0, head_radius - head_radius_penetration))
        .circle(head_radius)
        .extrude(height, both=True)
    )
    y_depth = arc_box.faces("<Y").val().Center().y
    return arc_box.cut(arc_cyclinder).translate((0, -y_depth))


def create_cutout_cylinder(
    radius: float,
    height: float,
    penetration: float,
):
    return (
        cq.Workplane("XY", origin=(0, radius - penetration))
        .circle(radius)
        .extrude(height / 2, both=True)
    )


def create_headbox(
    width: float,
    height: float,
    depth: float,
    head_radius: float,
    head_radius_penetration: float,
    board_hole_spacing: Tuple[float, float],
    through_hole_width: float,
    board_mount_clearance: float,
):
    arc = create_arc(
        width=width,
        height=height,
        head_radius=head_radius,
        head_radius_penetration=head_radius_penetration,
    )
    box = cq.Workplane("XZ").box(width, height, depth, centered=(True, True, False))
    cutout_cylinder = create_cutout_cylinder(
        radius=3,
        height=height / 2,
        penetration=0.75,
    )
    box = box.union(arc).cut(cutout_cylinder)
    through_hole = (
        cq.Workplane("XZ")
        .box(through_hole_width, height / 4, depth, centered=(True, True, False))
        .edges("|Y")
        .fillet(0.2)
    )
    box = box.cut(through_hole)
    strap_width = 0.5
    for strap_position in (-width / 2, width / 2 - strap_width):
        strap_guide = create_strap_guide(
            height=height / 4,
            width=strap_width,
            depth=0.4,
            strap_clearance=0.2,
        ).translate((strap_position, -depth, 0))
        box = box.union(strap_guide)
    board_hole_spacing_x, board_hole_spacing_z = board_hole_spacing
    board_hole_positions = [
        (board_hole_spacing_x / 2, board_hole_spacing_z / 2),
        (-board_hole_spacing_x / 2, board_hole_spacing_z / 2),
        (-board_hole_spacing_x / 2, -board_hole_spacing_z / 2),
        (board_hole_spacing_x / 2, -board_hole_spacing_z / 2),
    ]
    for x, z in board_hole_positions:
        box = box.union(
            create_screw_mount(
                hole_diameter=0.26,
                hole_padding=0.2,
                depth=board_mount_clearance,
            ).translate((x, -0.5, z))
        )
    return box


create_headbox(
    10,
    10,
    1,
    8.5,
    2,
    board_hole_spacing=(7.415, 7.9),
    through_hole_width=0.75,
    board_mount_clearance=1.25,
)

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…